# Rasterizar

In [1]:
# gdal_rasterize -a google_class -l 1-mvp -tr 10 10 -te 3298060 9564778 3299080 9565798 -ot Int16 -a_nodata -1 1-mvp.gpkg 1-mvp.tif

# gdalwarp -t_srs EPSG:32635 -te 624600 7192530 625060 7193000 -tr 10 10 -r mode -dstnodata -1 1-mvp.tif 1-mvp-warped.tif

In [ ]:
import shutil
from pathlib import Path

input_dir = Path("../../data/raw")
output_dir = Path("../../data/raster")
feature_dir = Path('../../data/feature')

output_dir.mkdir(exist_ok=True)
(output_dir / 'bing').mkdir(exist_ok=True)
(output_dir / 'google').mkdir(exist_ok=True)

gpkg_files = list(input_dir.glob("*.gpkg"))

In [3]:
import rasterio
import numpy as np
import geopandas as gpd

from tqdm import tqdm
from osgeo import gdal, osr

In [4]:
def rasterize(input_dir: Path, output_dir: Path, source: str):
    layer = input_dir.stem
    gdf = gpd.read_file(input_dir, layer=layer)
    xmin, ymin, xmax, ymax = gdf.total_bounds

    options = gdal.RasterizeOptions(
        attribute=f"{source}_value",
        layers=[layer],
        outputType=gdal.GDT_Int16,
        xRes=10,
        yRes=10,
        noData=-1,
        outputBounds=(xmin, ymin, xmax, ymax)
    )

    gdal.Rasterize(output_dir / f"{layer}.tif", input_dir, options=options)

In [ ]:
def get_output_bounds(ds):
    gt = ds.GetGeoTransform()
    xmin = gt[0]
    ymin = gt[3]
    xres = gt[1]
    yres = gt[5]
    xsize = ds.RasterXSize
    ysize = ds.RasterYSize
    xmax = xmin + xres * xsize
    ymax = ymin + yres * ysize

    return (xmin, ymin, xmax, ymax)

def get_epsg_from_gpkg(ds) -> int:
    proj = ds.GetProjection()
    srs = osr.SpatialReference()
    srs.ImportFromWkt(proj)

    return srs.GetAttrValue("AUTHORITY", 1)

def warp(input_dir: Path, output_dir: Path, source: str):
    input_fn = input_dir.stem

    ds = gdal.Open(feature_dir / source / f'{input_fn}.tif')

    warp_options = gdal.WarpOptions(
        format="GTiff",
        dstSRS=f"EPSG:{get_epsg_from_gpkg(ds)}",
        outputBounds=get_output_bounds(ds),
        xRes=10, yRes=10,                  
        resampleAlg=gdal.GRA_NearestNeighbour,           
        dstNodata=-1                       
    )

    gdal.Warp(output_dir / f'{input_fn}.tif', input_dir, options=warp_options)

In [ ]:
def mask(input_dir: Path, output_dir: Path, source: str):
    fn = input_dir.stem

    with rasterio.open(input_dir) as msrc:
        mask = msrc.read()
        mask = mask[:, ::-1, :]

    with rasterio.open(feature_dir / source / f'{fn}.tif') as fsrc:
        data = fsrc.read()
        profile = fsrc.profile.copy()

    nodata = profile.get("nodata")

    masked_data = np.where(np.isin(mask, [0, 1, 2, 3]), data, nodata)

    data_concat = np.concatenate([masked_data, mask], axis=0)

    profile.update(count=data_concat.shape[0])

    with rasterio.open(output_dir / f'{fn}.tif', "w", **profile) as dst:
        dst.write(data_concat)

In [7]:
rtemp_dir = Path("../data/temp_r")
rtemp_dir.mkdir(exist_ok=True)
(rtemp_dir / 'bing').mkdir(exist_ok=True)
(rtemp_dir / 'google').mkdir(exist_ok=True)

wtemp_dir = Path("../data/temp_w")
wtemp_dir.mkdir(exist_ok=True)
(wtemp_dir / 'bing').mkdir(exist_ok=True)
(wtemp_dir / 'google').mkdir(exist_ok=True)

In [8]:
for gpkg_file in tqdm(gpkg_files):
    for source in ['google', 'bing']:
        try:
            rasterize(gpkg_file, rtemp_dir / source, source)

            warp(rtemp_dir / source / f"{gpkg_file.stem}.tif", wtemp_dir / source, source)

            mask(wtemp_dir / source / f"{gpkg_file.stem}.tif", output_dir / source, source)
        except Exception as e:
            pass

  0%|          | 0/9799 [00:00<?, ?it/s]

c:\Users\tiago\anaconda3\envs\gdal\Lib\site-packages\osgeo\gdal.py:330: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
  0%|          | 10/9799 [00:04<1:15:26,  2.16it/s]


KeyboardInterrupt: 

In [ ]:
shutil.rmtree(rtemp_dir)
shutil.rmtree(wtemp_dir)

: 